# Installing Requirements

In [ ]:
pip install torch pandas scikit-learn transformers datasets joblib flask-ngrok

# Load & Inspect the Dataset

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("retail_store_inventory.csv")

df = df.dropna()

df["text"] = (
    "Product " + df["Product ID"].astype(str) +
    " category " + df["Category"].astype(str) +
    " region " + df["Region"].astype(str) +
    " season " + df["Seasonality"].astype(str)
)

df["demand_change"] = df["Demand Forecast"] - df["Units Sold"]

df["trend"] = np.where(
    df["demand_change"] > 5, "INCREASING",
    np.where(df["demand_change"] < -5, "DECREASING", "STABLE")
)

df[["Product ID", "Units Sold", "Demand Forecast", "trend"]].head()

,Product ID,Units Sold,Demand Forecast,trend
0,P0001,127,135.47,INCREASING
1,P0002,150,144.04,DECREASING
2,P0003,65,74.02,INCREASING
3,P0004,61,62.18,STABLE
4,P0005,14,9.26,STABLE


## Import Required Libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
encoder = AutoModel.from_pretrained(MODEL_NAME)

encoder.eval()

def embed_in_batches(texts, batch_size=32):
    embeddings = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]

            encoded = tokenizer(
                batch,
                padding=True,
                truncation=True,
                return_tensors="pt"
            )

            output = encoder(**encoded)
            batch_embeddings = output.last_hidden_state.mean(dim=1)
            embeddings.append(batch_embeddings.cpu())

    return torch.cat(embeddings, dim=0)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

# Loading the Text Embedding Model

In [ ]:
text_embeddings = embed_in_batches(
    df["text"].tolist(),
    batch_size=16
)

## Generating Text Embeddings

In [ ]:
numeric_features = torch.tensor(
    df[[
        "Inventory Level",
        "Units Sold",
        "Units Ordered",
        "Demand Forecast",
        "Price",
        "Discount",
        "Competitor Pricing"
    ]].values,
    dtype=torch.float32
)

X = torch.cat([text_embeddings, numeric_features], dim=1)

## Preparing Numeric Features

In [ ]:
import numpy as np

df["demand_change"] = df["Demand Forecast"] - df["Units Sold"]

df["trend"] = np.where(
    df["demand_change"] > 5, "INCREASING",
    np.where(df["demand_change"] < -5, "DECREASING", "STABLE")
)

label_map = {"DECREASING": 0, "STABLE": 1, "INCREASING": 2}
y = torch.tensor(df["trend"].map(label_map).values)

df[["Product ID", "Units Sold", "Demand Forecast", "trend"]].head()

,Product ID,Units Sold,Demand Forecast,trend
0,P0001,127,135.47,INCREASING
1,P0002,150,144.04,DECREASING
2,P0003,65,74.02,INCREASING
3,P0004,61,62.18,STABLE
4,P0005,14,9.26,STABLE


# Feature Engineering

In [ ]:
import torch.nn as nn

class DemandTrendModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(X.shape[1], 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        return self.net(x)

model = DemandTrendModel()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

## Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

for epoch in range(15):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = loss_fn(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 7.5502
Epoch 2, Loss: 4.4275
Epoch 3, Loss: 2.1035
Epoch 4, Loss: 1.4433
Epoch 5, Loss: 1.6250
Epoch 6, Loss: 2.0308
Epoch 7, Loss: 2.1629
Epoch 8, Loss: 2.1451
Epoch 9, Loss: 2.2179
Epoch 10, Loss: 2.2866
Epoch 11, Loss: 2.2462
Epoch 12, Loss: 2.1453
Epoch 13, Loss: 2.0718
Epoch 14, Loss: 2.0218
Epoch 15, Loss: 1.9138


# Model Training

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
with torch.no_grad():
    preds = model(X_test).argmax(dim=1)

print(classification_report(
    y_test.numpy(),
    preds.numpy(),
    target_names=["DECREASING", "STABLE", "INCREASING"]
))

print(confusion_matrix(y_test.numpy(), preds.numpy()))

              precision    recall  f1-score   support

  DECREASING       0.00      0.00      0.00      2389
      STABLE       0.35      0.49      0.41      4924
  INCREASING       0.53      0.55      0.54      7307

    accuracy                           0.44     14620
   macro avg       0.29      0.35      0.32     14620
weighted avg       0.38      0.44      0.41     14620

[[   0 1237 1152]
 [   0 2428 2496]
 [   0 3264 4043]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Setting Rules

In [ ]:
def stocking_rule(trend, inventory, forecast):
    if trend == "INCREASING" and inventory < forecast:
        return "INCREASE STOCK"
    elif trend == "INCREASING":
        return "PREPARE RESTOCK"
    elif trend == "STABLE":
        return "MAINTAIN STOCK"
    else:
        return "REDUCE STOCK"

# Model Evaluation and Predictions

In [ ]:
model.eval()
with torch.no_grad():
    all_preds = model(X).argmax(dim=1)

In [ ]:
inv_label_map = {0: "DECREASING", 1: "STABLE", 2: "INCREASING"}

df["predicted_trend"] = [inv_label_map[p.item()] for p in all_preds]

In [ ]:
df["decision"] = df.apply(
    lambda x: stocking_rule(
        x["predicted_trend"],
        x["Inventory Level"],
        x["Demand Forecast"]
    ),
    axis=1
)

df[["Product ID", "predicted_trend", "decision"]].head()

,Product ID,predicted_trend,decision
0,P0001,STABLE,MAINTAIN STOCK
1,P0002,INCREASING,PREPARE RESTOCK
2,P0003,INCREASING,PREPARE RESTOCK
3,P0004,INCREASING,PREPARE RESTOCK
4,P0005,STABLE,MAINTAIN STOCK


# Exporting Predictions to CSV

In [ ]:
columns_to_drop = ["text"]

df_export = df.drop(columns=columns_to_drop)

df_export.to_csv("inventory_predictions.csv", index=False)

In [ ]:
output_path = "inventory_predictions.csv"
df.to_csv(output_path, index=False)

print("Saved:", output_path)

Saved: inventory_predictions.csv


# Exporting Model

In [ ]:
torch.save(model.state_dict(), "trend_model.pt")

In [ ]:
from google.colab import files
files.download(output_path)
files.download("trend_model.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import joblib

label_map = {"DECREASING": 0, "STABLE": 1, "INCREASING": 2}
joblib.dump(label_map, "label_map.joblib")

['label_map.joblib']

In [ ]:
from google.colab import files
files.download("label_map.joblib")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>